<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mt5

!nvidia-smi

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

!pip install simpletransformers
!pip install torch
!pip install sacrebleu

In [ ]:
# config model

import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# load dataset
train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 512
model_args.max_length = 100
model_args.train_batch_size = 4
model_args.eval_batch_size = 4
model_args.num_train_epochs = 20
model_args.learning_rate = 4e-5
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "Machine Translation"

device = torch.cuda.is_available()

model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)
# website for more model
# https://huggingface.co/models?sort=downloads

In [ ]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

# Evaluate the model
result = model.eval_model(eval_df)

In [ ]:
# load model

import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs/best_model", args=model_args)

In [ ]:
# calculate bleu score

test = pd.read_csv("/content/gdrive/MyDrive/test.csv").astype(str)

thai_truth = [test.loc[test["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = test.loc[test["prefix"] == "translate english to thai"]["input_text"].tolist()
english_truth = [test.loc[test["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = test.loc[test["prefix"] == "translate thai to english"]["input_text"].tolist()

english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)
thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

In [ ]:
# predict test

to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
predictions = model.predict(to_predict)
for i, info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')